# PROGRAMA 2

## Objetivo:
- Implementación de una red neuronal para la detección de ciberacoso a partir de una recopilación de tweets (actualmente X's).erior uso.

## Tratamiento y visualización de los datos

In [1]:
#Importación de los datos
import pandas as pd
data_train = pd.read_csv('E:\MATERIAS CURSADAS\ONCEAVO SEMESTRE\PLN\cyberbullying_train.csv')
data_test = pd.read_csv('E:\MATERIAS CURSADAS\ONCEAVO SEMESTRE\PLN\cyberbullying_test.csv')
data_valid = pd.read_csv('E:\MATERIAS CURSADAS\ONCEAVO SEMESTRE\PLN\cyberbullying_val.csv')

data_train.head()

,tweet_text,cyberbullying_type
0,@AntonSirius @erinspice @prpltnkr @ChiefElk oh...,other_cyberbullying
1,"Ladies ""Bedroom Bully"" The Mix Cd By @GappyRan...",not_cyberbullying
2,RT @_bobbidana: Never thought I'd say this but...,not_cyberbullying
3,She is intellectual terrorists and world suffe...,religion
4,You saudias are not friends of Muslim idiots c...,religion


In [2]:
data_test.head()

,tweet_text,cyberbullying_type
0,"@dankmtl Yeap, as a little Propaganda Nazi for...",religion
1,@LyndseyBoo wow!! Leave lyndsey phone alone!!!...,other_cyberbullying
2,"I'm not a man, only know how to bully people :(",other_cyberbullying
3,@urgedharry @nyazpolitics @greenlinerzjm If yo...,religion
4,Calling people dumb isnt nice :( RT @tayyoung_...,ethnicity


In [3]:
data_valid.head()

,tweet_text,cyberbullying_type
0,But you idiot tagged and more also I'm a Muslim,religion
1,@SEXYLALA That's a dumb nigger never fuck LALA...,ethnicity
2,I never hear hispanics calling each other brow...,ethnicity
3,@Lenny_Banx nigger read it again such a dumb a...,ethnicity
4,She was a mean girl in high school. She and Mi...,age


In [4]:
#Bibliotecas necesarias
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Disable tensorflow debugging logs
os.environ["KERAS_BACKEND"] = "torch"
import keras
import torch

torch.__version__, keras.__version__

('2.4.1+cpu', '3.5.0')

- Clasificación en clases de los tipos de twwets que tenemos

In [5]:
#Uso de tf.data para la clasificación de clases a partir de vectores ONE-HOT
import tensorflow as tf

#Vectores ONE-HOT del conjunto de entrenamiento, prueba y validación
vector_train = data_train['cyberbullying_type']
vector_test = data_test['cyberbullying_type']
vector_val = data_valid['cyberbullying_type']

def mapeo_one_hot(vectores):
    categorias = list(set(vectores))
    valor_categorias = {vectores: index for index, vectores in enumerate(categorias)}

    indices = [valor_categorias[vector] for vector in vectores]

    one_hot = tf.one_hot(indices, depth=len(categorias))

    return one_hot

one_hot_train = mapeo_one_hot(vector_train)
one_hot_test = mapeo_one_hot(vector_test)
one_hot_val = mapeo_one_hot(vector_val)

print(one_hot_train.numpy())

[[1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 ...
 [0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]]


In [8]:
import tensorflow as tf
from keras.layers import TextVectorization
from sklearn.model_selection import train_test_split
import time

X_train = data_train['tweet_text'].values
y_train = one_hot_train

x_test = data_test['tweet_text'].values
y_test = one_hot_test

x_val = data_valid['tweet_text'].values
y_val = one_hot_val

In [9]:
len(X_train), len(x_test), len(x_val)

(33384, 7154, 7154)

In [10]:
raw_train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
raw_test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test))
raw_val_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val))

In [11]:
# Tamaño de vocabulario
vocab_size = 10000

vectorize_layer = TextVectorization(
    standardize=None,
    max_tokens=vocab_size,
    output_mode='tf-idf')

# Obtener el texto sin etiquetas
text_ds = raw_train_ds.map(lambda x, y: x)
start = time.time()
vectorize_layer.adapt(text_ds)
print(f'Time for adapt is {time.time()-start:.4f}')

Time for adapt is 183.0893


In [12]:
batch_size = 16
raw_train_ds = raw_train_ds.shuffle(20000).batch(batch_size)
raw_test_ds = raw_test_ds.batch(batch_size)
raw_val_ds = raw_val_ds.batch(batch_size)

In [13]:
for inp, target in raw_train_ds.take(1):
    print(inp[:2], target[:2])

tf.Tensor(
[b"Cannot believe my princess is 3 next! Where has time gone?! She's growing up too quick!:( love her so so much xxxxxxxx"
 b"RT @tauriqmoosa: Oh, is the Witcher 3 a game that let's you play as a lesbian of colour? That is remarkable! http://t.co/HfQLh0U04Z"], shape=(2,), dtype=string) tf.Tensor(
[[0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]], shape=(2, 6), dtype=float32)


In [14]:
vectorize_layer(inp)

tensor([[37.7287,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [37.7287,  1.2918,  2.7284,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [15.0915,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [45.2744,  1.2918,  1.3642,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [15]:
def preprocess(x, y):
    x = vectorize_layer(x)
    return x, y

train_ds = raw_train_ds.map(lambda x, y: preprocess(x, y))
val_ds = raw_val_ds.map(lambda x, y: preprocess(x, y))
test_ds = raw_test_ds.map(lambda x, y: preprocess(x, y))

In [16]:
next(iter(train_ds))

(<tf.Tensor: shape=(16, 10000), dtype=float32, numpy=
 array([[22.637196 ,  2.5835607,  0.       , ...,  0.       ,  0.       ,
          0.       ],
        [37.72866  ,  2.5835607,  1.3641971, ...,  0.       ,  0.       ,
          0.       ],
        [45.27439  ,  2.5835607,  1.3641971, ...,  0.       ,  0.       ,
          0.       ],
        ...,
        [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
          0.       ],
        [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
          0.       ],
        [15.091464 ,  0.       ,  0.       , ...,  0.       ,  0.       ,
          0.       ]], dtype=float32)>,
 <tf.Tensor: shape=(16, 6), dtype=float32, numpy=
 array([[0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0.],
        [1., 0., 

- Construcción del modelo de red neuronal.

In [17]:
class Linear(keras.layers.Layer):
    def __init__(self, num_outputs, activation=None):
        super().__init__()
        self.num_outputs = num_outputs
        self.activation = keras.activations.get(activation)

    def build(self, input_shape):
      input_dim = input_shape[-1]
      self.w = self.add_weight(shape=[input_dim,
                                      self.num_outputs],
                               name = 'kernel')
      self.b = self.add_weight(shape=[self.num_outputs],
                               name = 'bias')

    def call(self, inputs):
      x = keras.ops.matmul(inputs, self.w) + self.b
      return self.activation(x)

layer = Linear(6, 'tanh')
x = tf.ones([1, 10000])
out = layer(x)
out.shape

torch.Size([1, 6])

In [18]:
test_batch = next(iter(test_ds))
test_batch[0]

<tf.Tensor: shape=(16, 10000), dtype=float32, numpy=
array([[22.637196 ,  1.2917804,  1.3641971, ...,  0.       ,  0.       ,
         0.       ],
       [45.27439  ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  1.3641971, ...,  0.       ,  0.       ,
         0.       ],
       ...,
       [37.72866  ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [30.182928 ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 7.545732 ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ]], dtype=float32)>

In [21]:
class Model(keras.Model):
    def __init__(self, activation):
        super().__init__()
        self.l1 = Linear(3, activation=activation)
        self.l2 = Linear(6)

    def call(self, x):
        x = self.l1(x)
        x = self.l2(x)
        return x

model = Model('softmax')

In [22]:
model(test_batch[0]).shape, test_batch[1].shape

(torch.Size([16, 6]), TensorShape([16, 6]))

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Definir los callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3),  # Detener cuando la pérdida de validación no mejore por 3 épocas consecutivas.
    ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)  # Guardar el mejor modelo.
]

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=keras.losses.MeanSquaredError(),
    metrics=["accuracy"]
)

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=6,
    callbacks=callbacks
)

keras.layers.Dropout(0.1, noise_shape=None, seed=None, **kwargs)

model.evaluate(

)